In [ ]:
# pip install tensorflow opencv-python mediapipe sklearn matplotlib

In [161]:
import cv2
import mediapipe as mp
import numpy as np
import os
import time
from matplotlib import pyplot as plt

In [162]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [163]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [164]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 

In [165]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [219]:
import json

with open('archive/WLASL.json') as f:
    data = json.load(f)

counter2 = 0
for i in data:
    gloss = i['gloss']
    counter = 0
    for inst in i['instances']:
        videoID = inst['video_id']
        # check if video exists
        if os.path.isfile('archive/videos/' + videoID + '.mp4'):
            print("Video ID : ", videoID)
            with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
                cap = cv2.VideoCapture('archive/videos/' + videoID + '.mp4')
                frameRate = cap.get(2)
                filename = 0
                totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
                print("Total Frames: ", totalFrames)

                frameCounter = 0
                while (cap.isOpened()):
                    frameId = cap.get(1)
                    ret, frame = cap.read()
                    if ret != True:
                        break
                    if (frameId % ((int(frameRate)+1)) == 0):
                        image, results = mediapipe_detection(frame, holistic)
                        draw_styled_landmarks(image, results)
                        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                        keypoints = extract_keypoints(results)
                        frameId = int(frameId)
                        npy_path = os.path.join("train", gloss, str(counter) ,str(frameCounter) + ".npy")
                        if not os.path.exists(os.path.dirname(npy_path)):
                            os.makedirs(os.path.dirname(npy_path))
                        np.save(npy_path, keypoints)
                        frameCounter+=1
                counter+=1
        else:
            print("Video ID : ", videoID, " does not exist")
    
    if counter2 == 2:
        break
    counter2+=1
                    
    cap.release()
    cv2.destroyAllWindows()
    


Video ID :  69241
Total Frames:  75.0
Video ID :  65225  does not exist
Video ID :  68011  does not exist
Video ID :  68208  does not exist
Video ID :  68012  does not exist
Video ID :  70212  does not exist
Video ID :  70266  does not exist
Video ID :  07085  does not exist
Video ID :  07086  does not exist
Video ID :  07087  does not exist
Video ID :  07069
Total Frames:  30.0
Video ID :  07088  does not exist
Video ID :  07089  does not exist
Video ID :  07090  does not exist
Video ID :  07091  does not exist
Video ID :  07092  does not exist
Video ID :  07093  does not exist
Video ID :  07068
Total Frames:  68.0
Video ID :  07094  does not exist
Video ID :  07095  does not exist
Video ID :  07096  does not exist
Video ID :  07097  does not exist
Video ID :  07070
Total Frames:  86.0
Video ID :  07098  does not exist
Video ID :  07099
Total Frames:  87.0
Video ID :  07071  does not exist
Video ID :  07072  does not exist
Video ID :  07073  does not exist
Video ID :  67424  does not 

In [220]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [221]:
actions = np.array(["book", "drink", "computer"])
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'book': 0, 'drink': 1, 'computer': 2}

In [246]:
from keras.utils import pad_sequences
sequences, labels = [], []
sizeArr = []
DATA_PATH = os.path.join('train')
counter = 0

# create an array of size 1662 filled with -10
temp = np.full((1662), -10)

for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        frameCount = len(os.listdir(os.path.join(DATA_PATH, action, str(sequence))))
        for frameNum in range(frameCount):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), str(frameNum) + '.npy'))
            window.append(res)
            sizeArr.append(len(window))

        while (len(window) < 107):
            window.append(temp)
        sequences.append(window)
        labels.append(label_map[action])

maxVal = max(sizeArr)
print("Max Value : ",maxVal)


Max Value :  107


In [247]:
np.array(sequences).shape

(35, 107, 1662)

In [248]:
np.array(labels).shape

(35,)

In [249]:
X = np.array(sequences)

In [250]:
y = to_categorical(labels).astype(int)

In [251]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

Build and Train LSTM Neural Network

In [260]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import regularizers

In [253]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [261]:
# create a lstm model with backpropagation for continuous data
# model = Sequential()
# model.add(Masking(mask_value=-10, input_shape=(None, 1662)))
# model.add(LSTM(128, return_sequences=True))
# model.add(LSTM(128, return_sequences=True))
# model.add(LSTM(128, return_sequences=True))
# model.add(LSTM(128))
# model.add(Dense(actions.shape[0], activation='softmax'))


# model = Sequential()
# model.add(Masking(mask_value=-10, input_shape=(None, 1662)))
# model.add(LSTM(64, return_sequences=True, activation='relu'))
# model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(LSTM(64, return_sequences=False, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(16, activation='relu'))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(4, activation='relu'))
# model.add(Dense(2, activation='relu'))
# model.add(Dense(actions.shape[0], activation='softmax'))


# create a lstm model with backpropagation and regularization for continuous data
model = Sequential()
model.add(Masking(mask_value=-10, input_shape=(None, 1662)))
model.add(LSTM(64, return_sequences=True, activation='relu', kernel_regularizer=regularizers.l2(0.01), recurrent_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)))
model.add(LSTM(128, return_sequences=True, activation='relu', kernel_regularizer=regularizers.l2(0.01), recurrent_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)))
model.add(LSTM(64, return_sequences=False, activation='relu', kernel_regularizer=regularizers.l2(0.01), recurrent_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)))
model.add(Dense(actions.shape[0], activation='softmax'))

In [262]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [263]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
2/2 [==============================] - 4s 145ms/step - loss: 14.5389 - categorical_accuracy: 0.1515
Epoch 2/100
2/2 [==============================] - 0s 130ms/step - loss: 201.1411 - categorical_accuracy: 0.3939
Epoch 3/100
2/2 [==============================] - 0s 153ms/step - loss: 2075.1750 - categorical_accuracy: 0.4242
Epoch 4/100
2/2 [==============================] - 0s 125ms/step - loss: 31787.0078 - categorical_accuracy: 0.3030
Epoch 5/100
2/2 [==============================] - 0s 139ms/step - loss: 12367.8584 - categorical_accuracy: 0.3939
Epoch 6/100
2/2 [==============================] - 0s 149ms/step - loss: 54711.9688 - categorical_accuracy: 0.4545
Epoch 7/100
2/2 [==============================] - 0s 145ms/step - loss: 937979.0000 - categorical_accuracy: 0.3636
Epoch 8/100
2/2 [==============================] - 0s 145ms/step - loss: 184303.0625 - categorical_accuracy: 0.3030
Epoch 9/100
2/2 [==============================] - 1s 147ms/step - loss: 1162967.250

In [188]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_2 (Masking)         (None, None, 1662)        0         
                                                                 
 lstm_27 (LSTM)              (None, None, 64)          442112    
                                                                 
 lstm_28 (LSTM)              (None, None, 128)         98816     
                                                                 
 lstm_29 (LSTM)              (None, 64)                49408     
                                                                 
 dense_27 (Dense)            (None, 64)                4160      
                                                                 
 dense_28 (Dense)            (None, 32)                2080      
                                                                 
 dense_29 (Dense)            (None, 3)                

In [268]:
res = model.predict(X_test)

1/1 [==============================] - 0s 60ms/step


In [269]:
model.save('action.h5')

In [270]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [271]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 59ms/step


In [272]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [273]:
accuracy_score(ytrue, yhat)

0.5

In [274]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        print("Start")

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-107:]

        if len(sequence) == 107:
            print("Predicting")
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            print(predictions)
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()